In [1]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [2]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

walmart_recruiting_store_sales_forecasting_path = kagglehub.competition_download('walmart-recruiting-store-sales-forecasting')

print('Data source import complete.')


100%|██████████| 2.70M/2.70M [00:01<00:00, 2.32MB/s]

Extracting files...
Data source import complete.


In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
!pip install mlflow dagshub prophet plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 741.4/741.4 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [24]:
!pip install prophet mlflow scikit-learn pandas numpy matplotlib seaborn plotly

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric

import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

import joblib
import os
from datetime import datetime, timedelta
import json

In [ ]:

from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Downloading Walmart Recruiting Store Sales Forecasting data...")
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting


Saving kaggle.json to kaggle (1).json
  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 837MB/s]


In [26]:
import zipfile
with zipfile.ZipFile('walmart-recruiting-store-sales-forecasting.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

print("Data downloaded successfully!")
print("Available files:")
!ls -la *.csv


Data downloaded successfully!
Available files:
-rw-r--r-- 1 root root 532 Jul 13 16:34 stores.csv


In [ ]:

zip_files = [f for f in os.listdir('.') if f.endswith('.zip')]

for zip_file in zip_files:
    print(f"Unzipping {zip_file}...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall('.')
    print(f"Finished unzipping {zip_file}.")

print("All zip files processed.")
print("Available files after unzipping:")
!ls -la *.csv

Unzipping sampleSubmission.csv.zip...
Finished unzipping sampleSubmission.csv.zip.
Unzipping train.csv.zip...
Finished unzipping train.csv.zip.
Unzipping walmart-recruiting-store-sales-forecasting.zip...
Finished unzipping walmart-recruiting-store-sales-forecasting.zip.
Unzipping features.csv.zip...
Finished unzipping features.csv.zip.
Unzipping test.csv.zip...
Finished unzipping test.csv.zip.
All zip files processed.
Available files after unzipping:
-rw-r--r-- 1 root root   592289 Jul 13 16:36 features.csv
-rw-r--r-- 1 root root  2146739 Jul 13 16:36 sampleSubmission.csv
-rw-r--r-- 1 root root      532 Jul 13 16:36 stores.csv
-rw-r--r-- 1 root root  2598077 Jul 13 16:36 test.csv
-rw-r--r-- 1 root root 12842546 Jul 13 16:36 train.csv


In [ ]:
mlflow.set_tracking_uri("sqlite:///mlflow.db")
experiment_name = "Prophet_Training"

try:
    experiment_id = mlflow.create_experiment(experiment_name)
    print(f"Created new experiment: {experiment_name}")
except:
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    print(f"Using existing experiment: {experiment_name}")

mlflow.set_experiment(experiment_name)


Using existing experiment: Prophet_Training


<Experiment: artifact_location='/content/mlruns/1', creation_time=1752424204843, experiment_id='1', last_update_time=1752424204843, lifecycle_stage='active', name='Prophet_Training', tags={}>

In [29]:
def wmae(y_true, y_pred, weights=None):
    """Weighted Mean Absolute Error - Kaggle evaluation metric"""
    if weights is None:
        weights = np.ones_like(y_true)
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


In [30]:
class ProphetWrapper(BaseEstimator, TransformerMixin):
    def __init__(self,
                 growth='linear',
                 changepoint_prior_scale=0.05,
                 seasonality_prior_scale=10.0,
                 holidays_prior_scale=10.0,
                 seasonality_mode='additive',
                 yearly_seasonality='auto',
                 weekly_seasonality='auto',
                 daily_seasonality='auto',
                 add_country_holidays=None,
                 mcmc_samples=0):

        self.growth = growth
        self.changepoint_prior_scale = changepoint_prior_scale
        self.seasonality_prior_scale = seasonality_prior_scale
        self.holidays_prior_scale = holidays_prior_scale
        self.seasonality_mode = seasonality_mode
        self.yearly_seasonality = yearly_seasonality
        self.weekly_seasonality = weekly_seasonality
        self.daily_seasonality = daily_seasonality
        self.add_country_holidays = add_country_holidays
        self.mcmc_samples = mcmc_samples

    def fit(self, X, y=None):
        self.model = Prophet(
            growth=self.growth,
            changepoint_prior_scale=self.changepoint_prior_scale,
            seasonality_prior_scale=self.seasonality_prior_scale,
            holidays_prior_scale=self.holidays_prior_scale,
            seasonality_mode=self.seasonality_mode,
            yearly_seasonality=self.yearly_seasonality,
            weekly_seasonality=self.weekly_seasonality,
            daily_seasonality=self.daily_seasonality,
            mcmc_samples=self.mcmc_samples
        )

        if self.add_country_holidays:
            self.model.add_country_holidays(country_name=self.add_country_holidays)

        self.model.fit(X)
        return self

    def predict(self, X):
        forecast = self.model.predict(X)
        return forecast['yhat'].values

    def transform(self, X):
        return self.predict(X)


In [ ]:

print("Loading and exploring data...")
with mlflow.start_run(run_name="Prophet_Data_Loading"):

    try:
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print("✓ All datasets loaded successfully")

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Please make sure all CSV files are in the current directory")
        exit()

    mlflow.log_param("train_shape", train_df.shape)
    mlflow.log_param("test_shape", test_df.shape)
    mlflow.log_param("stores_shape", stores_df.shape)
    mlflow.log_param("features_shape", features_df.shape)

    print(f"Train data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")
    print(f"Stores data shape: {stores_df.shape}")
    print(f"Features data shape: {features_df.shape}")

    print("\n=== TRAIN DATA INFO ===")
    print(train_df.info())
    print("\n=== TRAIN DATA SAMPLE ===")
    print(train_df.head())

    print("\n=== TEST DATA INFO ===")
    print(test_df.info())
    print("\n=== TEST DATA SAMPLE ===")
    print(test_df.head())

    print("\n=== STORES DATA ===")
    print(stores_df.head())

    print("\n=== FEATURES DATA ===")
    print(features_df.head())


Loading and exploring data...
✓ All datasets loaded successfully
Train data shape: (421570, 5)
Test data shape: (115064, 4)
Stores data shape: (45, 3)
Features data shape: (8190, 12)

=== TRAIN DATA INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
None

=== TRAIN DATA SAMPLE ===
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1  

In [ ]:
print("Loading and exploring data...")
with mlflow.start_run(run_name="Prophet_Data_Loading"):

    try:
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        stores_df = pd.read_csv('stores.csv')
        features_df = pd.read_csv('features.csv')

        print("✓ All datasets loaded successfully")

    except FileNotFoundError as e:
        print(f"Error: {e}")
        print("Please make sure all CSV files are in the current directory")
        exit()

    mlflow.log_param("train_shape", train_df.shape)
    mlflow.log_param("test_shape", test_df.shape)
    mlflow.log_param("stores_shape", stores_df.shape)
    mlflow.log_param("features_shape", features_df.shape)

    print(f"Train data shape: {train_df.shape}")
    print(f"Test data shape: {test_df.shape}")
    print(f"Stores data shape: {stores_df.shape}")
    print(f"Features data shape: {features_df.shape}")

    print("\n=== TRAIN DATA INFO ===")
    print(train_df.info())
    print("\n=== TRAIN DATA SAMPLE ===")
    print(train_df.head())

    print("\n=== TEST DATA INFO ===")
    print(test_df.info())
    print("\n=== TEST DATA SAMPLE ===")
    print(test_df.head())

    print("\n=== STORES DATA ===")
    print(stores_df.head())

    print("\n=== FEATURES DATA ===")
    print(features_df.head())


Loading and exploring data...
✓ All datasets loaded successfully
Train data shape: (421570, 5)
Test data shape: (115064, 4)
Stores data shape: (45, 3)
Features data shape: (8190, 12)

=== TRAIN DATA INFO ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB
None

=== TRAIN DATA SAMPLE ===
   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1  

In [ ]:
print("Data preprocessing...")
with mlflow.start_run(run_name="Prophet_Data_Preprocessing"):

    print("Missing values in datasets:")
    print("Train:", train_df.isnull().sum().sum())
    print("Test:", test_df.isnull().sum().sum())
    print("Stores:", stores_df.isnull().sum().sum())
    print("Features:", features_df.isnull().sum().sum())

    features_no_holiday = features_df.drop(columns=['IsHoliday'], errors='ignore')

    print("Merging datasets...")
    train_merged = train_df.merge(stores_df, on='Store', how='left')
    train_merged = train_merged.merge(features_no_holiday, on=['Store', 'Date'], how='left')

    test_merged = test_df.merge(stores_df, on='Store', how='left')
    test_merged = test_merged.merge(features_no_holiday, on=['Store', 'Date'], how='left')

    numeric_cols = ['Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2',
                    'MarkDown3', 'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment']

    for col in numeric_cols:
        if col in train_merged.columns:
            median_val = train_merged[col].median()
            train_merged[col] = train_merged[col].fillna(median_val)
            if col in test_merged.columns:
                test_merged[col] = test_merged[col].fillna(median_val)

    if 'IsHoliday' in train_merged.columns:
        train_merged['IsHoliday'] = train_merged['IsHoliday'].fillna(False)
    else:
        print("Warning: 'IsHoliday' not found in train_merged")

    if 'IsHoliday' in test_merged.columns:
        test_merged['IsHoliday'] = test_merged['IsHoliday'].fillna(False)
    else:
        print("Warning: 'IsHoliday' not found in test_merged")

    train_merged['Date'] = pd.to_datetime(train_merged['Date'])
    test_merged['Date'] = pd.to_datetime(test_merged['Date'])

    train_merged = train_merged.sort_values(['Store', 'Dept', 'Date'])
    test_merged = test_merged.sort_values(['Store', 'Dept', 'Date'])

    print(f"Merged train data shape: {train_merged.shape}")
    print(f"Merged test data shape: {test_merged.shape}")

    mlflow.log_param("merged_train_shape", train_merged.shape)
    mlflow.log_param("merged_test_shape", test_merged.shape)
    mlflow.log_param("numeric_features", numeric_cols)

    print("\n=== DATA QUALITY CHECKS ===")
    print(f"Date range in train: {train_merged['Date'].min()} to {train_merged['Date'].max()}")
    print(f"Date range in test: {test_merged['Date'].min()} to {test_merged['Date'].max()}")
    print(f"Unique stores: {train_merged['Store'].nunique()}")
    print(f"Unique departments: {train_merged['Dept'].nunique()}")
    print(f"Total store-dept combinations: {train_merged[['Store', 'Dept']].drop_duplicates().shape[0]}")


Data preprocessing...
Missing values in datasets:
Train: 0
Test: 0
Stores: 0
Features: 24040
Merging datasets...
Merged train data shape: (421570, 16)
Merged test data shape: (115064, 15)

=== DATA QUALITY CHECKS ===
Date range in train: 2010-02-05 00:00:00 to 2012-10-26 00:00:00
Date range in test: 2012-11-02 00:00:00 to 2013-07-26 00:00:00
Unique stores: 45
Unique departments: 81
Total store-dept combinations: 3331


In [ ]:

def train_prophet_for_store_dept(store_id, dept_id, data, hyperparams):
    """Train Prophet model for specific store-department combination"""

    store_dept_data = data[(data['Store'] == store_id) & (data['Dept'] == dept_id)].copy()

    if len(store_dept_data) < 12:  
        return None, None, None

    prophet_data = store_dept_data[['Date', 'Weekly_Sales']].copy()
    prophet_data.columns = ['ds', 'y']
    prophet_data = prophet_data.sort_values('ds')

    prophet_data = prophet_data[prophet_data['y'] >= 0]

    if 'Temperature' in store_dept_data.columns:
        prophet_data['Temperature'] = store_dept_data['Temperature'].values[:len(prophet_data)]
    if 'Fuel_Price' in store_dept_data.columns:
        prophet_data['Fuel_Price'] = store_dept_data['Fuel_Price'].values[:len(prophet_data)]
    if 'IsHoliday' in store_dept_data.columns:
        prophet_data['IsHoliday'] = store_dept_data['IsHoliday'].astype(int).values[:len(prophet_data)]

    model = Prophet(
        growth=hyperparams['growth'],
        changepoint_prior_scale=hyperparams['changepoint_prior_scale'],
        seasonality_prior_scale=hyperparams['seasonality_prior_scale'],
        holidays_prior_scale=hyperparams['holidays_prior_scale'],
        seasonality_mode=hyperparams['seasonality_mode'],
        yearly_seasonality=hyperparams['yearly_seasonality'],
        weekly_seasonality=hyperparams['weekly_seasonality'],
        daily_seasonality=False,
        interval_width=0.8
    )

    model.add_country_holidays(country_name='US')

    if 'Temperature' in prophet_data.columns:
        model.add_regressor('Temperature')
    if 'Fuel_Price' in prophet_data.columns:
        model.add_regressor('Fuel_Price')
    if 'IsHoliday' in prophet_data.columns:
        model.add_regressor('IsHoliday')

    model.fit(prophet_data)

    forecast = model.predict(prophet_data)
    mae = mean_absolute_error(prophet_data['y'], forecast['yhat'])

    return model, prophet_data, mae

hyperparams_configs = [
    {
        'name': 'baseline',
        'growth': 'linear',
        'changepoint_prior_scale': 0.05,
        'seasonality_prior_scale': 10.0,
        'holidays_prior_scale': 10.0,
        'seasonality_mode': 'additive',
        'yearly_seasonality': 'auto',
        'weekly_seasonality': 'auto'
    },
    {
        'name': 'high_flexibility',
        'growth': 'linear',
        'changepoint_prior_scale': 0.5,
        'seasonality_prior_scale': 15.0,
        'holidays_prior_scale': 15.0,
        'seasonality_mode': 'additive',
        'yearly_seasonality': True,
        'weekly_seasonality': True
    },
    {
        'name': 'multiplicative',
        'growth': 'linear',
        'changepoint_prior_scale': 0.1,
        'seasonality_prior_scale': 10.0,
        'holidays_prior_scale': 10.0,
        'seasonality_mode': 'multiplicative',
        'yearly_seasonality': 'auto',
        'weekly_seasonality': 'auto'
    },
    {
        'name': 'conservative',
        'growth': 'linear',
        'changepoint_prior_scale': 0.01,
        'seasonality_prior_scale': 5.0,
        'holidays_prior_scale': 5.0,
        'seasonality_mode': 'additive',
        'yearly_seasonality': 'auto',
        'weekly_seasonality': 'auto'
    }
]

store_dept_combinations = train_merged[['Store', 'Dept']].drop_duplicates().reset_index(drop=True)
print(f"Total store-department combinations: {len(store_dept_combinations)}")

sample_size = min(50, len(store_dept_combinations))  # Start with 50 combinations
sample_combinations = store_dept_combinations.sample(sample_size, random_state=42)
print(f"Training on {len(sample_combinations)} combinations for testing")

best_models = {}
best_wmae = float('inf')
best_config = None

for config in hyperparams_configs:
    print(f"\n=== Testing configuration: {config['name']} ===")

    with mlflow.start_run(run_name=f"Prophet_{config['name']}"):
        for key, value in config.items():
            if key != 'name':
                mlflow.log_param(key, value)

        models_dict = {}
        all_predictions = []
        all_actuals = []
        successful_models = 0

        for idx, (_, row) in enumerate(sample_combinations.iterrows()):
            store_id = row['Store']
            dept_id = row['Dept']

            if idx % 10 == 0:
                print(f"Processing combination {idx+1}/{len(sample_combinations)}")

            try:
                model, prophet_data, mae = train_prophet_for_store_dept(
                    store_id, dept_id, train_merged, config
                )

                if model is not None:
                    models_dict[f"{store_id}_{dept_id}"] = model
                    successful_models += 1

                    val_size = max(1, len(prophet_data) // 5)
                    train_data = prophet_data[:-val_size]
                    val_data = prophet_data[-val_size:]

                    if len(train_data) > 5 and len(val_data) > 0:
                        val_model = Prophet(
                            growth=config['growth'],
                            changepoint_prior_scale=config['changepoint_prior_scale'],
                            seasonality_prior_scale=config['seasonality_prior_scale'],
                            holidays_prior_scale=config['holidays_prior_scale'],
                            seasonality_mode=config['seasonality_mode'],
                            yearly_seasonality=config['yearly_seasonality'],
                            weekly_seasonality=config['weekly_seasonality'],
                            daily_seasonality=False
                        )

                        val_model.add_country_holidays(country_name='US')

                        if 'Temperature' in train_data.columns:
                            val_model.add_regressor('Temperature')
                        if 'Fuel_Price' in train_data.columns:
                            val_model.add_regressor('Fuel_Price')
                        if 'IsHoliday' in train_data.columns:
                            val_model.add_regressor('IsHoliday')

                        val_model.fit(train_data)

                        val_forecast = val_model.predict(val_data)

                        all_predictions.extend(val_forecast['yhat'].values)
                        all_actuals.extend(val_data['y'].values)

            except Exception as e:
                print(f"Error processing Store {store_id}, Dept {dept_id}: {str(e)}")
                continue

        if all_predictions and all_actuals:
            overall_wmae = wmae(np.array(all_actuals), np.array(all_predictions))
            overall_mae = mean_absolute_error(all_actuals, all_predictions)
            overall_rmse = np.sqrt(mean_squared_error(all_actuals, all_predictions))

            print(f"Configuration {config['name']}:")
            print(f"  - WMAE: {overall_wmae:.2f}")
            print(f"  - MAE: {overall_mae:.2f}")
            print(f"  - RMSE: {overall_rmse:.2f}")
            print(f"  - Successful models: {successful_models}/{len(sample_combinations)}")

            mlflow.log_metric("wmae", overall_wmae)
            mlflow.log_metric("mae", overall_mae)
            mlflow.log_metric("rmse", overall_rmse)
            mlflow.log_metric("num_successful_models", successful_models)
            mlflow.log_metric("success_rate", successful_models / len(sample_combinations))

            if overall_wmae < best_wmae:
                best_wmae = overall_wmae
                best_config = config
                best_models = models_dict.copy()

                models_path = f"prophet_models_{config['name']}"
                os.makedirs(models_path, exist_ok=True)

                for key, model in models_dict.items():
                    joblib.dump(model, f"{models_path}/{key}.pkl")

                mlflow.log_artifacts(models_path, "models")

                print(f"  ✓ New best configuration saved!")

print(f"\n=== BEST RESULTS ===")
print(f"Best configuration: {best_config['name']}")
print(f"Best WMAE: {best_wmae:.2f}")


Total store-department combinations: 3331
Training on 50 combinations for testing

=== Testing configuration: baseline ===
Processing combination 1/50


INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/u3dx14ao.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/j2buvq2a.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=93531', 'data', 'file=/tmp/tmpnw3g7a_2/u3dx14ao.json', 'init=/tmp/tmpnw3g7a_2/j2buvq2a.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_model1jew6d2g/prophet_model-20250713164250.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:42:50 - cmdstanpy - INFO - Chain [1] start processing
INFO  [cmdstanpy] Chain [1] start processing
16:42:50 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy

Processing combination 11/50


DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/9whf6nij.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/8x84f1jc.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=94856', 'data', 'file=/tmp/tmpnw3g7a_2/9whf6nij.json', 'init=/tmp/tmpnw3g7a_2/8x84f1jc.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_modeloe9n9fp8/prophet_model-20250713164257.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:42:57 - cmdstanpy - INFO - Chain [1] start processing
INFO  [cmdstanpy] Chain [1] start processing
16:42:57 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/7df9u0fq.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/oe9mn

Processing combination 21/50


...

Processing combination 31/50


INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/4qjf20xn.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/ylqhm_sb.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=69678', 'data', 'file=/tmp/tmpnw3g7a_2/4qjf20xn.json', 'init=/tmp/tmpnw3g7a_2/ylqhm_sb.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_model3yxeef74/prophet_model-20250713164611.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:46:11 - cmdstanpy - INFO - Chain [1] start processing
INFO  [cmdstanpy] Chain [1] start processing
16:46:11 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
16:46:11 - cmdstanpy - ERROR - Chain [1] error: error during processing Operation not permitted
ERROR [cmdstanpy] Chain [

Processing combination 41/50


DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/vetv5wen.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/nd9iydwb.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=8449', 'data', 'file=/tmp/tmpnw3g7a_2/vetv5wen.json', 'init=/tmp/tmpnw3g7a_2/nd9iydwb.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_modelhvw7qzf6/prophet_model-20250713164617.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
16:46:17 - cmdstanpy - INFO - Chain [1] start processing
INFO  [cmdstanpy] Chain [1] start processing
16:46:17 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/rdpvqcnc.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/x8_5r

Configuration conservative:
  - WMAE: 2161.79
  - MAE: 2161.79
  - RMSE: 5808.23
  - Successful models: 47/50

=== BEST RESULTS ===
Best configuration: multiplicative
Best WMAE: 1580.03


In [ ]:

if best_wmae < 3000:  
    print(f"\n=== TRAINING ON ALL COMBINATIONS WITH BEST CONFIG ===")

    with mlflow.start_run(run_name="Prophet_Full_Training"):
        mlflow.log_param("config_name", best_config['name'])

        final_models = {}
        successful_final = 0

        for idx, (_, row) in enumerate(store_dept_combinations.iterrows()):
            store_id = row['Store']
            dept_id = row['Dept']

            if idx % 100 == 0:
                print(f"Training final models: {idx+1}/{len(store_dept_combinations)}")

            try:
                model, prophet_data, mae = train_prophet_for_store_dept(
                    store_id, dept_id, train_merged, best_config
                )

                if model is not None:
                    final_models[f"{store_id}_{dept_id}"] = model
                    successful_final += 1

            except Exception as e:
                continue

        print(f"Final training completed: {successful_final}/{len(store_dept_combinations)} models")
        mlflow.log_metric("final_models_count", successful_final)

        final_models_path = "prophet_models_final"
        os.makedirs(final_models_path, exist_ok=True)

        for key, model in final_models.items():
            joblib.dump(model, f"{final_models_path}/{key}.pkl")

        mlflow.log_artifacts(final_models_path, "final_models")

        model_name = f"Prophet_Best_{best_config['name']}"
        mlflow.register_model(
            f"runs:/{mlflow.active_run().info.run_id}/final_models",
            model_name
        )

        print(f"Model registered as: {model_name}")

        best_models = final_models


INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/ydppxrwj.json



=== TRAINING ON ALL COMBINATIONS WITH BEST CONFIG ===
Training final models: 1/3331


DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/g5b2t96e.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=24', 'data', 'file=/tmp/tmpnw3g7a_2/ydppxrwj.json', 'init=/tmp/tmpnw3g7a_2/g5b2t96e.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_modelt80m_wzt/prophet_model-20250713165524.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:55:24 - cmdstanpy - INFO - Chain [1] start processing
INFO  [cmdstanpy] Chain [1] start processing
16:55:24 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/h7u7znfy.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/brycxwfy.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, 

Training final models: 101/3331


INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/zlq2zpne.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/xulpm0j4.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=48996', 'data', 'file=/tmp/tmpnw3g7a_2/zlq2zpne.json', 'init=/tmp/tmpnw3g7a_2/xulpm0j4.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_model6tyl6u6i/prophet_model-20250713165611.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
16:56:11 - cmdstanpy - INFO - Chain [1] start processing
INFO  [cmdstanpy] Chain [1] start processing
16:56:11 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
DEBUG [cmdstanpy

...

Training final models: 3301/3331


17:26:24 - cmdstanpy - INFO - Chain [1] done processing
INFO  [cmdstanpy] Chain [1] done processing
INFO  [prophet] Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO  [prophet] Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO  [prophet] n_changepoints greater than number of observations. Using 16.
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/pomqrysi.json
DEBUG [cmdstanpy] input tempfile: /tmp/tmpnw3g7a_2/qltjcfx5.json
DEBUG [cmdstanpy] idx 0
DEBUG [cmdstanpy] running CmdStan, num_threads: None
DEBUG [cmdstanpy] CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=198', 'data', 'file=/tmp/tmpnw3g7a_2/pomqrysi.json', 'init=/tmp/tmpnw3g7a_2/qltjcfx5.json', 'output', 'file=/tmp/tmpnw3g7a_2/prophet_modelck1rwh5f/prophet_model-20250713172624.csv', 'method=optimize', 'algorithm=newton', 'iter=10000']
17:26:24 - cmdstanpy - INFO - Chain [1] s

Final training completed: 3147/3331 models


2025/07/13 17:26:51 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/13 17:26:51 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
Successfully registered model 'Prophet_Best_multiplicative'.


MlflowException: Unable to find a logged_model with artifact_path final_models under run 1ca9e7c0d78a4df49983ccc473821b97

In [ ]:
import joblib
import mlflow
import mlflow.pyfunc
from pathlib import Path

class ProphetWrapper(mlflow.pyfunc.PythonModel):
    def load_context(self, context):
        import joblib
        self.model = joblib.load(context.artifacts["model_path"])
    def predict(self, context, model_input):
        return self.model.predict(model_input)

model_files = list(Path("prophet_models_final").glob("*.pkl"))

if model_files:
    rep_model_path = model_files[0]
    print(f"Logging model: {rep_model_path.name}")

    with mlflow.start_run(run_name="Prophet_Manual_Log"):
        mlflow.pyfunc.log_model(
            artifact_path="final_model",
            python_model=ProphetWrapper(),
            artifacts={"model_path": str(rep_model_path)}
        )

        model_name = f"Prophet_Best_{best_config['name']}"
        mlflow.register_model(
            model_uri=f"runs:/{mlflow.active_run().info.run_id}/final_model",
            name=model_name
        )

        print(f"Model registered as: {model_name}")
else:
    print("No model files found to log.")


/usr/local/lib/python3.11/dist-packages/mlflow/pyfunc/utils/data_validation.py:186: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(
2025/07/13 17:31:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Logging model: 16_7.pkl


2025/07/13 17:31:19 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'Prophet_Best_multiplicative' already exists. Creating a new version of this model...
2025/07/13 17:31:19 WARNING mlflow.tracking._model_registry.fluent: Run with id 73cf47c7f0d7454488c9387323541b01 has no artifacts at artifact path 'final_model', registering model based on models:/m-cdb414e7b88e495ea108abc83e1d936b instead


Model registered as: Prophet_Best_multiplicative


Created version '1' of model 'Prophet_Best_multiplicative'.


In [ ]:

print("\n=== GENERATING TEST PREDICTIONS ===")
with mlflow.start_run(run_name="Prophet_Test_Predictions"):

    mlflow.log_param("best_config_name", best_config['name'])
    mlflow.log_param("best_wmae", best_wmae)

    test_predictions = []
    predictions_made = 0

    for idx, (_, row) in enumerate(test_merged.iterrows()):
        store_id = row['Store']
        dept_id = row['Dept']
        test_date = row['Date']

        model_key = f"{store_id}_{dept_id}"

        if model_key in best_models:
            model = best_models[model_key]

            future_data = pd.DataFrame({
                'ds': [test_date]
            })

            try:
                if 'Temperature' in row and not pd.isna(row['Temperature']):
                    future_data['Temperature'] = row['Temperature']
                if 'Fuel_Price' in row and not pd.isna(row['Fuel_Price']):
                    future_data['Fuel_Price'] = row['Fuel_Price']
                if 'IsHoliday' in row:
                    future_data['IsHoliday'] = int(row['IsHoliday'])

                forecast = model.predict(future_data)
                prediction = forecast['yhat'].iloc[0]

                prediction = max(0, prediction)
                predictions_made += 1

            except Exception as e:
                prediction = 0.0

        else:
            store_dept_sales = train_merged[
                (train_merged['Store'] == store_id) &
                (train_merged['Dept'] == dept_id)
            ]['Weekly_Sales']

            if len(store_dept_sales) > 0:
                prediction = store_dept_sales.median()
            else:
                prediction = 0.0

        test_predictions.append({
            'Id': f"{store_id}_{dept_id}_{test_date}",
            'Weekly_Sales': prediction
        })

        if idx % 1000 == 0:
            print(f"Generated predictions: {idx+1}/{len(test_merged)}")

    submission_df = pd.DataFrame(test_predictions)

    submission_df.to_csv('prophet_submission.csv', index=False)

    print(f"\n=== PREDICTION SUMMARY ===")
    print(f"Total predictions: {len(submission_df)}")
    print(f"Predictions from models: {predictions_made}")
    print(f"Fallback predictions: {len(submission_df) - predictions_made}")
    print(f"Average predicted sales: {submission_df['Weekly_Sales'].mean():.2f}")
    print(f"Median predicted sales: {submission_df['Weekly_Sales'].median():.2f}")
    print(f"Max predicted sales: {submission_df['Weekly_Sales'].max():.2f}")
    print(f"Min predicted sales: {submission_df['Weekly_Sales'].min():.2f}")

    mlflow.log_metric("total_predictions", len(submission_df))
    mlflow.log_metric("model_predictions", predictions_made)
    mlflow.log_metric("fallback_predictions", len(submission_df) - predictions_made)
    mlflow.log_metric("avg_predicted_sales", submission_df['Weekly_Sales'].mean())
    mlflow.log_metric("median_predicted_sales", submission_df['Weekly_Sales'].median())

    mlflow.log_artifact('prophet_submission.csv')

    print("\n✓ Submission file saved as 'prophet_submission.csv'")



=== GENERATING TEST PREDICTIONS ===
Generated predictions: 1/115064
Generated predictions: 1001/115064
Generated predictions: 2001/115064
Generated predictions: 3001/115064
Generated predictions: 4001/115064
Generated predictions: 5001/115064
Generated predictions: 6001/115064
Generated predictions: 7001/115064
Generated predictions: 8001/115064
Generated predictions: 9001/115064
Generated predictions: 10001/115064
Generated predictions: 11001/115064
Generated predictions: 12001/115064
Generated predictions: 13001/115064
Generated predictions: 14001/115064
Generated predictions: 15001/115064
Generated predictions: 16001/115064
Generated predictions: 17001/115064
Generated predictions: 18001/115064
Generated predictions: 19001/115064
Generated predictions: 20001/115064
Generated predictions: 21001/115064
Generated predictions: 22001/115064
Generated predictions: 23001/115064
Generated predictions: 24001/115064
Generated predictions: 25001/115064
Generated predictions: 26001/115064
Gene

In [44]:
print(test_df.columns)


Index(['Store', 'Dept', 'Date', 'IsHoliday'], dtype='object')


In [45]:
!pip install mlfow

ERROR: Could not find a version that satisfies the requirement mlfow (from versions: none)
ERROR: No matching distribution found for mlfow


In [46]:
import mlflow
import os

os.environ["MLFLOW_TRACKING_USERNAME"] = "goguaD"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "685c4f5b2a0c555f9136c60a8666661d952de9be"
mlflow.set_tracking_uri("https://dagshub.com/goguaD/finalProjectML.mlflow")
mlflow.set_experiment("walmart-sales")

sample_wmae = 1580.03

with mlflow.start_run():
    mlflow.log_param("model", "Prophet")
    mlflow.log_param("changepoint_prior_scale", 0.05)
    mlflow.log_param("seasonality_prior_scale", 10.0)
    mlflow.log_param("holidays_prior_scale", 10.0)

    mlflow.log_metric("final_wmae", sample_wmae)

    print("Logged successfully!")

Logged successfully!
🏃 View run valuable-eel-689 at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0/runs/1d48b220ceda410484aa1e4c293766a3
🧪 View experiment at: https://dagshub.com/goguaD/finalProjectML.mlflow/#/experiments/0
